In [1]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.llms import LlamaCpp
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from IPython.display import display, HTML
from langchain.chains import LLMChain
import json
import time
import pathlib

### **Consume Information from the Documents (.txt)**

In [ ]:
# define what documents to load
loader = DirectoryLoader("./data/", glob="*.txt", loader_cls=TextLoader)

# interpret information in the documents
documents = loader.load()
splitter = RecursiveCharacterTextSplitter()
texts = splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'})

# create and save the local database
db = FAISS.from_documents(texts, embeddings)
db.save_local("faiss")

#### **Prepare Template**

In [2]:
template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
Question: {question}
Only return the helpful answer below and nothing else.
Helpful answer:
"""

#### **load the interpreted information from the local database**

In [3]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'})
db = FAISS.load_local("faiss", embeddings)

/media/mdrafsunsheikh/10784ce9-f796-443d-b973-fe6ae114c687/home/Study/Projects/llama_cpp/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# prepare a version of the llm pre-loaded with the local content
retriever = db.as_retriever(search_kwargs={'k': 2})

In [5]:
# prompt = PromptTemplate(template=template, input_variables=['context', 'question'])
prompt = PromptTemplate(
    template=template,
    input_variables=['context', 'question'])

In [6]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

### **Load the Model**

In [7]:
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    # model_path="./llama.cpp/models/quantized_q4_1.gguf",
    model_path="./model/llama-2-7b-chat.gguf.q4_0.bin",
    temperature=0.01,
    max_tokens=1000,
    top_p=1,
    n_ctx=6000,
    # callback_manager=callback_manager,
    # verbose=True,  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./model/llama-2-7b-chat.gguf.q4_0.bin (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = llama-2-7b-chat.ggmlv3.q4_0.bin
llama_model_loader: - kv   2:                        general.description str              = converted from legacy GGJTv3 MOSTLY_Q...
llama_model_loader: - kv   3:                          general.file_type u32              = 2
llama_model_loader: - kv   4:                       llama.context_length u32              = 2048
llama_model_loader: - kv   5:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   6:                          llama.block_count u32              = 32
llama_model_loade

#### **Model Retrieval Chain and Query Function**

In [9]:
QA_LLM = RetrievalQA.from_chain_type(llm=llm,
                                     chain_type='stuff',
                                     retriever=retriever,
                                     return_source_documents=True,
                                     chain_type_kwargs={'prompt': prompt})

def query(model, question):
    # model_path = model.combine_documents_chain.llm_chain.llm.model
    # model_name = pathlib.Path(model_path).name
    time_start = time.time()
    output = model({'query': question})
    response = output["result"]
    time_elapsed = time.time() - time_start
    # display(HTML(f'<code>{model_name} response time: {time_elapsed:.02f} sec</code>'))
    display(HTML(f'<strong>Question:</strong> {question}'))
    display(HTML(f'<strong>Answer:</strong> {response}'))
    # print(output)

#### **Ask your question Here**

In [12]:
query(QA_LLM, "What is equity?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =   34637.86 ms
llama_print_timings:      sample time =      62.28 ms /    83 runs   (    0.75 ms per token,  1332.65 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   33262.77 ms /    83 runs   (  400.76 ms per token,     2.50 tokens per second)
llama_print_timings:       total time =   33676.86 ms
